# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# 06 — Session & Memory Roundtrip (Configurable, Real Adapters)

Goals:
- Verify that DropInKnowledgeRuntime can:
  - create a new session (when session_id is None),
  - reuse an existing session (when session_id is provided),
  - persist and load conversation history via SessionManager.get_history(...),
  - produce a consistent debug_trace["steps"].

Notes:
- This notebook uses real adapters (no fakes, no fallbacks).
- Edit only the "Runtime configuration" cell to switch LLM / embeddings / vector store implementations.
- RAG / websearch / tools are disabled in this notebook to keep the baseline stable.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
# ==========================================================
# RUNTIME CONFIGURATION
# ==========================================================

from intergrax.globals.settings import GLOBAL_SETTINGS
from intergrax.llm_adapters.ollama_adapter import LangChainOllamaAdapter
from intergrax.rag.embedding_manager import EmbeddingManager


# --- Adapter ---
llm_adapter = LangChainOllamaAdapter()


# --- Embeddings ---
embed_manager = EmbeddingManager(
    verbose=True,
    provider="ollama",
    model_name=GLOBAL_SETTINGS.default_ollama_embed_model, 
    assume_ollama_dim=1536)

# --- Vector store ---
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager

vectorstore_manager = VectorstoreManager(
    config = VSConfig(
        provider="chroma",
        collection_name="intergrax_docs"
    )
)

# --- Feature flags ---
ENABLE_RAG = False
ENABLE_WEBSEARCH = False
ENABLE_TOOLS = False


TypeError: LangChainOllamaAdapter.__init__() missing 1 required positional argument: 'chat'